In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
import pandas as pd
from pprint import pprint
import openweathermapy.core as owm
import seaborn as sns

In [2]:
import numpy as np

np_lat = np.arange(-90,90,10)
np_lng = np.arange(-180,180,18)
# coordinates = (np_lat,np_lng) #Better keeping lat/lng in separate columns

import random

rnd_lat = random.choice(np_lat)
rnd_lng = random.choice(np_lng)
# rnd_coordinates = random.choice(coordinates) #Better keeping lat/lng in separate columns

from citipy import citipy

weather_df = pd.DataFrame()
weather_df['rnd_lat']=""
weather_df['rnd_lng']=""

for x in np_lat:
    for y in np_lng:
        x_values = list(np.arange(-90,90,0.01))
        y_values = list(np.arange(-180,180,0.01))
        lats = random.sample(x_values,17999) #why 17999 instead of 500? Because of duplicates..
        lons = random.sample(y_values,17999) #..stemming from the geocoords from citipy
                                            #and that's the maximum number of samples I can input
                                            #given the latitude parameters (180/0.01 = 18000)
                                            #Also I believe a larger sampling size to be better to
                                            #increase the random distribution of values.
        
weather_df = weather_df.append(pd.DataFrame.from_dict({"rnd_lat":lats,
                                       "rnd_lng":lons}))

weather_df = weather_df.reset_index(drop=True)



weather_df['city']=""
weather_df['country_code']=""

for index,row in weather_df.iterrows():
        weather_df_city = citipy.nearest_city(row["rnd_lat"], row["rnd_lng"])
        weather_df.set_value(index,"city",weather_df_city.city_name)
        weather_df.set_value(index,"country_code",weather_df_city.country_code)
        
weather_df=weather_df.drop(['rnd_lat', 'rnd_lng'],axis=1) #geo coord don't correspond to cities

weather_df=weather_df.drop_duplicates() #hmmm, some coords might yield similar cities....
print("Sampling size (from n = 17,999) after dumping duplicates: n=" + str(len(weather_df)))

weather_df = weather_df.sample(800) #resample so that n=800 (why? because some cities will may lack
                                    # data for certain attributes, so we'll re-sample once data has
                                    # dropped any rows with NULL values for any attributes)
weather_df = weather_df.reset_index(drop=True) #Remove index pre-resampling

print(weather_df.head())

print("Confirm n = 800: " + str(weather_df.shape))

C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:43: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Sampling size (from n = 17,999) after dumping duplicates: n=3412
          city country_code
0         cham           de
1  southbridge           nz
2        goure           ne
3     asbestos           ca
4   moses lake           us
Confirm n = 800: (800, 2)


In [5]:
from config2 import api_key 

    # Set parameters to pass in query URL
units = "metric" 
params = {
    "appid": api_key,
    "units": "metric",
}

    # Set counter for request iteration for each city
counter = 0

    # Set Request Loop
for index, row in weather_df.iterrows():
    params["q"] =f'{row["city"]},{row["country_code"]}' #Set location params: city, country_code
    target_url = 'http://api.openweathermap.org/data/2.5/weather?' #i.e. OWM's base API url

    counter += 1 # add to the index count for each iterations
#     print("Now retrieving city # " + str(counter)) #making sure the counter iteration is working
    
    city_owm = requests.get(target_url, params=params)
    print("ID:"+ str(counter) + " " + "OWM info for " + str( params["q"]))
    print(city_owm.url) #making sure the request iterations are working for each location.
    
    city_owm  = city_owm.json()
    
    #Pull attributes of interest from OWM request
    weather_df.set_value(index,"Latitude",city_owm.get("coord",{}).get("lat"))
    weather_df.set_value(index,"Longitude",city_owm.get("coord",{}).get("lon"))
    weather_df.set_value(index,"Temperature",city_owm.get("main",{}).get("temp_max"))
    weather_df.set_value(index,"Wind_speed",city_owm.get("wind",{}).get("speed"))
    weather_df.set_value(index,"Humidity",city_owm.get("main",{}).get("humidity"))
    weather_df.set_value(index,"Cloudiness",city_owm.get("clouds",{}).get("all"))

ID:1 OWM info for bowen,au
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=bowen%2Cau


C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:29: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packa

ID:2 OWM info for zaraza,ve
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=zaraza%2Cve
ID:3 OWM info for muscat,om
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=muscat%2Com
ID:4 OWM info for mayo,ca
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=mayo%2Cca
ID:5 OWM info for richards bay,za
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=richards+bay%2Cza
ID:6 OWM info for binzhou,cn
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=binzhou%2Ccn
ID:7 OWM info for capela,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=capela%2Cbr
ID:8 OWM info for bulawayo,zw
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=bulawayo%2Czw

ID:59 OWM info for pilar,ph
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=pilar%2Cph
ID:60 OWM info for maine-soroa,ne
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=maine-soroa%2Cne
ID:61 OWM info for port shepstone,za
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=port+shepstone%2Cza
ID:62 OWM info for saint augustine,us
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=saint+augustine%2Cus
ID:63 OWM info for batemans bay,au
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=batemans+bay%2Cau
ID:64 OWM info for peace river,ca
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=peace+river%2Cca
ID:65 OWM info for pontes e lacerda,br
http://api.openweathermap.org/data/2.5/weathe

ID:115 OWM info for chokurdakh,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=chokurdakh%2Cru
ID:116 OWM info for krishnarajpet,in
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=krishnarajpet%2Cin
ID:117 OWM info for iquitos,pe
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=iquitos%2Cpe
ID:118 OWM info for karlskrona,se
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=karlskrona%2Cse
ID:119 OWM info for savonlinna,fi
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=savonlinna%2Cfi
ID:120 OWM info for san andres,co
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=san+andres%2Cco
ID:121 OWM info for vila praia de ancora,pt
http://api.openweathermap.org/data/2.5/weather?app

...

ID:227 OWM info for ko samui,th
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=ko+samui%2Cth
ID:228 OWM info for workington,gb
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=workington%2Cgb
ID:229 OWM info for maniitsoq,gl
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=maniitsoq%2Cgl
ID:230 OWM info for zheleznodorozhnyy,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=zheleznodorozhnyy%2Cru
ID:231 OWM info for zatoka,ua
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=zatoka%2Cua
ID:232 OWM info for papara,pf
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=papara%2Cpf
ID:233 OWM info for pangoa,pe
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32

ID:283 OWM info for tonantins,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=tonantins%2Cbr
ID:284 OWM info for gornopravdinsk,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=gornopravdinsk%2Cru
ID:285 OWM info for taseyevo,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=taseyevo%2Cru
ID:286 OWM info for zastron,za
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=zastron%2Cza
ID:287 OWM info for kasempa,zm
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=kasempa%2Czm
ID:288 OWM info for gurupa,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=gurupa%2Cbr
ID:289 OWM info for santa elena,mx
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c1169

ID:338 OWM info for dongfeng,cn
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=dongfeng%2Ccn
ID:339 OWM info for valente,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=valente%2Cbr
ID:340 OWM info for nanning,cn
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=nanning%2Ccn
ID:341 OWM info for dudinka,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=dudinka%2Cru
ID:342 OWM info for djibo,bf
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=djibo%2Cbf
ID:343 OWM info for zlynka,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=zlynka%2Cru
ID:344 OWM info for mozarlandia,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&

ID:394 OWM info for kabo,cf
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=kabo%2Ccf
ID:395 OWM info for barrow,us
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=barrow%2Cus
ID:396 OWM info for kerewan,gm
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=kerewan%2Cgm
ID:397 OWM info for san francisco de becerra,hn
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=san+francisco+de+becerra%2Chn
ID:398 OWM info for vanderhoof,ca
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=vanderhoof%2Cca
ID:399 OWM info for guaimaro,cu
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=guaimaro%2Ccu
ID:400 OWM info for correntina,br
http://api.openweathermap.org/data/2.5/weather?appid=9f8748261

ID:450 OWM info for shebalino,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=shebalino%2Cru
ID:451 OWM info for petropavlovsk-kamchatskiy,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=petropavlovsk-kamchatskiy%2Cru
ID:452 OWM info for devils lake,us
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=devils+lake%2Cus
ID:453 OWM info for ilo,pe
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=ilo%2Cpe
ID:454 OWM info for mednogorsk,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=mednogorsk%2Cru
ID:455 OWM info for katima mulilo,na
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=katima+mulilo%2Cna
ID:456 OWM info for aksu,cn
http://api.openweathermap.org/data/2.5/weath

In [13]:
weather_df = weather_df.dropna()
# weather_df = weather_df.sample(500) #resample so that n=500
print(weather_df.shape)
weather_df.head()


# import csv
weather_df.to_csv("owm_sample.csv",index=False)

# Save to df
weather_csv = "owm_sample.csv"
weather_df = pd.read_csv(weather_csv)

(455, 8)


In [120]:
weather_df.to_pickle("weather_df.pkl")
weather_df=pd.read_pickle("weather_df.pkl")
print(weather_df.columns)
print(weather_df.shape)
weather_df.head()

weather_df_lat = weather_df['Latitude']
weather_df_temp = weather_df['Temperature']
weather_df_hum = weather_df.Humidity
weather_df_clo = weather_df.Cloudiness
weather_df_wind = weather_df.Wind_speed

df = pd.DataFrame({'Latitude': weather_df.Latitude,
                   'Temperature (°C)': weather_df.Temperature,
                   'Wind speed (mph)': weather_df.Wind_speed,
                  'Humidity (%)': weather_df.Humidity,
                   'Cloudiness (%)': weather_df.Cloudiness
                  })

df.head()

Index(['city', 'country_code', 'Latitude', 'Longitude', 'Temperature',
       'Wind_speed', 'Humidity', 'Cloudiness'],
      dtype='object')
(455, 8)


,Cloudiness (%),Humidity (%),Latitude,Temperature (°C),Wind speed (mph)
0,44.0,78.0,-20.01,25.00,4.60
1,8.0,62.0,9.35,28.19,5.26
2,0.0,41.0,23.60,25.00,2.60
3,90.0,36.0,63.59,7.00,1.00
4,20.0,96.0,-28.77,18.41,1.61


In [129]:
# Histogram of the distribution of sampled Latitudes
    # Plotly tutorial: https://plot.ly/python/histograms/

print(weather_df_lat.describe())
    
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

data = [go.Histogram(y=weather_df_lat)]
#######
    # Reference: https://plot.ly/python/shapes/

layout = {
    'xaxis': {
        'range': [0, 40]
    },
    'yaxis': {
        'range': [-50, 90]
    },
    'shapes': [

# Line Horizontal
        # Equator
        {
            'type': 'line',
            'x0': 0,
            'y0': 0,
            'x1': 40,
            'y1': 0,
            'line': {
                'color': 'rgb(220, 20, 60)',
                'width': 2
            },
        },
    ]}

fig = {
    'data': data,
    'layout': layout,
}
####
py.iplot(fig, filename='horizontal histogram')

count    455.000000
mean      21.457538
std       28.927899
min      -44.490000
25%       -2.650000
50%       26.020000
75%       45.220000
max       78.220000
Name: Latitude, dtype: float64


In [106]:
# Histogram of the distribution of sampled Temperatures

print(weather_df_temp.describe())

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

data = [go.Histogram(x=weather_df_temp)]

py.iplot(data, filename='basic histogram')

count    455.000000
mean      16.292154
std       10.220038
min      -21.320000
25%       10.000000
50%       18.190000
75%       24.685000
max       38.000000
Name: Temperature, dtype: float64


In [70]:
# Import packages for Regression Plot

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
from sklearn import datasets, linear_model

In [95]:
# Temperature (C) vs. Latitude

weather_df_temp=weather_df_temp.reshape(-1, 1)
weather_df_lat=weather_df_lat.reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_temp)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_temp) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_temp))

####

def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k
####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_temp, 
                mode='markers',
                marker=dict(color='black')
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                line=dict(color='blue', width=3)
                )

layout = go.Layout(xaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1, p2], layout=layout)

py.iplot(fig)

C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:3: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



Coefficients: 
 [[-0.22413937]]
Mean squared error: 62.27
Variance score: 0.40


In [107]:
# Humidity (%) vs. Latitude

print(weather_df_hum.describe())

weather_df_hum=weather_df_hum.reshape(-1, 1)
weather_df_lat=weather_df_lat.reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_hum)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_hum) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_hum))

####
def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k

####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_hum, 
                mode='markers',
                marker=dict(color='black')
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                line=dict(color='blue', width=3)
                )

layout = go.Layout(xaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1, p2], layout=layout)

py.iplot(fig)

count    455.000000
mean      73.718681
std       21.444734
min        7.000000
25%       60.000000
50%       78.000000
75%       92.000000
max      100.000000
Name: Humidity, dtype: float64
Coefficients: 
 [[-0.18456233]]
Mean squared error: 430.42
Variance score: 0.06


C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:5: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:6: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [108]:
# Cloudiness (%) vs. Latitude

print(weather_df_clo.describe())

weather_df_clo=weather_df_clo.reshape(-1, 1)
weather_df_lat=weather_df_lat.reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_clo)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_clo) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_clo))

####
def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k

####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_clo, 
                mode='markers',
                marker=dict(color='black')
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                line=dict(color='blue', width=3)
                )

layout = go.Layout(xaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1, p2], layout=layout)

py.iplot(fig)

count    455.000000
mean      35.492308
std       34.979343
min        0.000000
25%        0.000000
50%       24.000000
75%       75.000000
max      100.000000
Name: Cloudiness, dtype: float64
Coefficients: 
 [[0.13110055]]
Mean squared error: 1206.51
Variance score: 0.01


C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:5: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [109]:
# Wind Speed (mph) vs. Latitude

print(weather_df_wind.describe())

weather_df_wind=weather_df_wind.reshape(-1, 1)
weather_df_lat=weather_df_lat.reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_wind)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_wind) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_wind))

####
def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k

####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_wind, 
                mode='markers',
                marker=dict(color='black')
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                line=dict(color='blue', width=3)
                )

layout = go.Layout(xaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1, p2], layout=layout)

py.iplot(fig)

count    455.000000
mean       3.079714
std        2.167310
min        0.260000
25%        1.500000
50%        2.510000
75%        4.100000
max       12.900000
Name: Wind_speed, dtype: float64
Coefficients: 
 [[0.0175796]]
Mean squared error: 4.43
Variance score: 0.06


C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:5: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

